In [1]:
import pandas as pd
import os
import threading
import time
import numpy as np
import yfinance as yf

In [2]:
a_i_directory_path = r"C:\Users\Blake Dennett\Downloads\Spring2023\appliedProgramming\Data\stock_market_data\sp500\csv"
with_slash = r"C:\Users\Blake Dennett\Downloads\Spring2023\appliedProgramming\Data\stock_market_data\sp500\csv\'"

d_i_directory_path = r"C:\Users\Blake Dennett\Downloads\Spring2023\appliedProgramming\Data\stock_market_data\sp500\csv\d-i"
with_slash4 = r"C:\Users\Blake Dennett\Downloads\Spring2023\appliedProgramming\Data\stock_market_data\sp500\csv\d-i\'"

j_p_directory_path = r"C:\Users\Blake Dennett\Downloads\Spring2023\appliedProgramming\Data\stock_market_data\sp500\csv\j-p"
with_slash2 = r"C:\Users\Blake Dennett\Downloads\Spring2023\appliedProgramming\Data\stock_market_data\sp500\csv\j-p\'"

q_z_directory_path = r"C:\Users\Blake Dennett\Downloads\Spring2023\appliedProgramming\Data\stock_market_data\sp500\csv\j-p\q-z"
with_slash3 = r"C:\Users\Blake Dennett\Downloads\Spring2023\appliedProgramming\Data\stock_market_data\sp500\csv\j-p\q-z\'"

df = pd.DataFrame()
df2 = pd.DataFrame()
df3 = pd.DataFrame()
df4 = pd.DataFrame()
dataframes = []

In [3]:
# function to combine two datasets
def combine(dat1, dat2):
    to_combine = [dat1, dat2]
    return pd.concat(to_combine)


def get_company_name(id):
    msft = yf.Ticker(id)

    company_name = msft.info['longName']
    return company_name

In [4]:
# function to loop through and open files
def files_to_dataframe(directory_path, with_slash, df):
    for filename in os.listdir(directory_path):
        if filename == 'j-p'or filename == 'q-z' or filename == 'd-i':
            continue
        directory = with_slash[:-1]
        path = directory + filename
        with open(path, 'r') as file:
            dat = pd.read_csv(file)
        dat['company_id'] = filename[:len(filename)-4]
        df = combine(df, dat)
    dataframes.append(df)

# loops through the folder broken up into four threads that run concurently 
thread1 = threading.Thread(target=files_to_dataframe, args=(a_i_directory_path, with_slash, df))
thread2 = threading.Thread(target=files_to_dataframe, args=(j_p_directory_path, with_slash2, df2))
thread3 = threading.Thread(target=files_to_dataframe, args=(q_z_directory_path, with_slash3, df3))
thread4 = threading.Thread(target=files_to_dataframe, args=(d_i_directory_path, with_slash4, df4))

thread1.start()
thread2.start()
thread3.start()
thread4.start()
thread1.join()
thread2.join()
thread3.join()
thread4.join()

# combining the dataframes from each of the four threads
df = combine(dataframes[0], dataframes[1])
df2 = combine(dataframes[2], dataframes[3])
df = combine(df, df2)
print(len(df))

# df['company'] = df.apply(get_company_name, axis=1)
# df.drop(columns=['company_id'])

3266000
--- 59.45264220237732 seconds ---


# What is the biggest difference in the low and high?

In [5]:
# ==================================================== DATA ANALYSIS =========================================

df['difference'] = df.apply(lambda x: x.High - x.Low, axis=1)

max = df["difference"].max()
df.set_index("difference", inplace=True)

high = df.loc[max,'High']
low = df.loc[max,'Low']
company = get_company_name(df.loc[max,'company_id'])
date = df.loc[max, 'Date']

print(f"The biggest difference in high to low is {max} from {company}")
print(f'The high was {high:.2f} and the low was {low:.2f}')
print(f'The date was {date}')

df.reset_index(inplace=True)

The biggest difference in high to low is 53156.0 from Berkshire Hathaway Inc.
The high was 468795.00 and the low was 415639.00
The date was 27-10-2022
--- 212.78131747245789 seconds ---


# What is the largest difference by percentage?

In [7]:
df['percent_difference'] = df.apply(lambda x: (x.difference / x.High) * 100, axis=1)
max = df['percent_difference'].max()

df.set_index("percent_difference", inplace=True)
company = df.loc[max, 'company_id']
high = df.loc[max, 'High']
low = df.loc[max, 'Low']
date = df.loc[max, 'Date']

print(f"The biggest difference by percent from high to low is {max} from {get_company_name(company)}")
print(f'The high was {high:.2f} and the low was {low:.4f}')
print(f'The date was {date}')

df.reset_index(inplace=True)
# print(df.head())

The biggest difference by percent from high to low is 99.98492462247178 from Formcap Corp.
The high was 1.99 and the low was 0.0003
The date was 23-11-2020


# what is the oldest and newest date?

In [11]:
df['Date'] = pd.to_datetime(df['Date'], format="%m/%d/%Y")
df.head()

,percent_difference,difference,Date,Low,Open,Volume,High,Close,Adjusted Close,company_id
0,5.540352,3.988998,2015-02-01,68.010002,69.000000,4218700.0,71.999001,70.400002,70.400002,QRVO
1,5.946712,4.240005,2015-05-01,67.059998,71.230003,4534700.0,71.300003,67.629997,67.629997,QRVO
2,7.309895,4.969997,2015-06-01,63.020000,67.930000,3749400.0,67.989998,64.669998,64.669998,QRVO
3,4.868190,3.250004,2015-07-01,63.509998,65.010002,2122600.0,66.760002,66.650002,66.650002,QRVO
4,3.673418,2.560005,2015-08-01,67.129997,67.949997,2510300.0,69.690002,67.690002,67.690002,QRVO


In [ ]:
import datetime

dt = datetime.datetime.now() # replace with your own datetime object

def make_date_time(row):
    pass

seconds_since_epoch = (dt - datetime.datetime(1970, 1, 1)).total_seconds()
days_since_epoch = seconds_since_epoch / (24 * 60 * 60)
days_float = float(days_since_epoch)